In [1]:
import torchvision
print(torchvision.__version__)

0.21.0+cu124


In [2]:
from utils import Config
from data_loader import MultiOmicsDataset, create_dataloaders

In [3]:
# # Reload all modules every time before executing code
%load_ext autoreload
%autoreload 2  

### 7. Run pytest

In [4]:
%%bash
pytest -v


============================= test session starts ==============================
platform linux -- Python 3.11.11, pytest-8.3.5, pluggy-1.5.0 -- /home/CBBI/wangh5/miniforge3/envs/cs7643_a4_generative/bin/python3.11
cachedir: .pytest_cache
rootdir: /home/CBBI/wangh5/_PyCharm/proj_dl
configfile: pytest.ini
plugins: anyio-4.9.0
collecting ... collected 5 items

tests/test_02_model_components.py::test_vae_forward PASSED               [ 20%]
tests/test_02_model_components.py::test_mirna_vae_forward PASSED         [ 40%]
tests/test_02_model_components.py::test_mini_convnext PASSED             [ 60%]
tests/test_02_model_components.py::test_transformer_fusion PASSED        [ 80%]
tests/test_02_model_components.py::test_fusion_classifier PASSED         [100%]

============================== 5 passed in 0.28s ===============================


In [5]:
import torch
from utils import Config
from data_loader import MultiOmicsDataset, create_dataloaders
from models import MultiOmicsClassifier 

In [14]:
import json
from pathlib import Path

In [6]:
from trainers import BaseTrainer
from losses import MultiOmicsLoss

In [16]:
def get_training_parameters(trainer, include_model_info=False, include_optimizer_state=False):
    """Extracts all relevant training parameters in a structured dictionary.
    
    Args:
        trainer: BaseTrainer instance
        include_model_info: Whether to include model architecture details
        include_optimizer_state: Whether to include optimizer state details
        
    Returns:
        Dictionary containing all training parameters
    """
    params = {
        "training": {
            "device": str(trainer.device),
        },
        "loss": {
            "type": type(trainer.loss_fn).__name__,
            "beta": getattr(trainer.loss_fn, 'target_beta', None),
            "use_focal": getattr(trainer.loss_fn, 'use_focal', None),
            "focal_gamma": getattr(trainer.loss_fn, 'focal_gamma', None),
            "label_smoothing": getattr(trainer.loss_fn, 'label_smoothing', None),
            "kl_epsilon": getattr(trainer.loss_fn, 'kl_epsilon', None)
        },
        "optimizer": {
            "type": type(trainer.optimizer).__name__,
            "lr": trainer.optimizer.param_groups[0]['lr'],
            "betas": trainer.optimizer.param_groups[0].get('betas', None),
            "eps": trainer.optimizer.param_groups[0].get('eps', None),
            "weight_decay": trainer.optimizer.param_groups[0].get('weight_decay', None)
        }
    }
    
    if include_model_info:
        params["model"] = {
            "type": type(trainer.model).__name__,
            "total_parameters": sum(p.numel() for p in trainer.model.parameters()),
            "trainable_parameters": sum(p.numel() for p in trainer.model.parameters() 
                                      if p.requires_grad),
            "architecture": str(trainer.model)  # This shows the model structure
        }
    
    if include_optimizer_state:
        params["optimizer"]["state"] = {
            "momentum_buffer": any('momentum_buffer' in p for p in trainer.optimizer.state.values())
        }

    
    return params



# params = get_training_parameters(trainer)
# print(json.dumps(params, indent=4))


In [7]:
config = Config.from_yaml("configs/data_config.yaml")

dataset = MultiOmicsDataset(config)
dataloaders = create_dataloaders(dataset, config)



In [8]:
batch_size = 4
mirna_dim = 1046
rna_exp_dim = 13054
methy_shape = (50, 100)  # e.g., (50, 100)
latent_dim = 64
num_classes = 5

# Instantiate model
multiomics_model = MultiOmicsClassifier(
    mirna_dim=mirna_dim,
    rna_exp_dim=rna_exp_dim,
    methy_shape=methy_shape,
    latent_dim=latent_dim,
    num_classes=num_classes
)

In [22]:
trainer = BaseTrainer(
    model=multiomics_model,
    optimizer=torch.optim.Adam(multiomics_model.parameters(), lr=1e-4),
    loss_fn=MultiOmicsLoss(),
    device='cuda'
)

trainer.fit(train_loader=dataloaders['train'], val_loader=dataloaders['val'], epochs=10)



Epoch 1/10


Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 11.82it/s, loss=0.215]


Train Loss: 0.2023


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.80it/s]


Val Loss:   0.8421

Epoch 2/10


...

Val Loss:   0.4508

Epoch 10/10


Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 12.23it/s, loss=0.122]


Train Loss: 0.1498


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.71it/s]

Val Loss:   0.5895


In [23]:
preds, targets = trainer.predict(dataloaders["test"])

# Compute accuracy
from sklearn.metrics import accuracy_score

acc = accuracy_score(targets.numpy(), preds.numpy())
print(f"✅ Test Accuracy: {acc:.4f}")




✅ Test Accuracy: 0.6212


In [24]:
from sklearn.metrics import classification_report
print(classification_report(targets.numpy(), preds.numpy()))

params = get_training_parameters(trainer)
print(json.dumps(params, indent=4))


              precision    recall  f1-score   support

           0       0.55      1.00      0.71        29
           1       1.00      0.13      0.24        15
           2       1.00      0.22      0.36         9
           3       0.86      0.67      0.75         9
           4       1.00      0.50      0.67         4

    accuracy                           0.62        66
   macro avg       0.88      0.50      0.54        66
weighted avg       0.78      0.62      0.56        66

{
    "training": {
        "device": "cuda"
    },
    "loss": {
        "type": "MultiOmicsLoss",
        "beta": 0.1,
        "use_focal": false,
        "focal_gamma": 2.0,
        "label_smoothing": 0.0,
        "kl_epsilon": 1e-08
    },
    "optimizer": {
        "type": "Adam",
        "lr": 0.0001,
        "betas": [
            0.9,
            0.999
        ],
        "eps": 1e-08,
        "weight_decay": 0
    }
}


In [72]:
trainer = BaseTrainer(
    model=multiomics_model,
    optimizer=torch.optim.Adam(multiomics_model.parameters(), lr=2e-5),
    loss_fn=MultiOmicsLoss(
                use_focal=False,
                # focal_gamma=2,        # Reduced from 0.5 for gentler hard example focus
                beta=1,               # Lower KL divergence weight
                # label_smoothing=0.1,    # Added smoothing
                class_weights=None  # torch.tensor([1.0, 1.5, 1.0, 1.0, 2.0], device="cuda")  # Targeted rebalancing
),
    device='cuda'
)

trainer.fit(train_loader=dataloaders['train'], val_loader=dataloaders['val'], epochs=10)


Epoch 1/10


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.48it/s, loss=0.0298]


Train Loss: 0.0211


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.58it/s]


Val Loss:   0.5941

Epoch 2/10


...

Val Loss:   0.4237

Epoch 10/10


Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 11.44it/s, loss=0.147]


Train Loss: 0.1356


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.91it/s]

Val Loss:   0.6193


In [73]:
preds, targets = trainer.predict(dataloaders["test"])
acc = accuracy_score(targets.numpy(), preds.numpy())
print(f"✅ Test Accuracy: {acc:.4f}")

print(classification_report(targets.numpy(), preds.numpy()))
params = get_training_parameters(trainer)

print(json.dumps(params, indent=4))



✅ Test Accuracy: 0.8333
              precision    recall  f1-score   support

           0       0.89      0.83      0.86        29
           1       0.68      0.87      0.76        15
           2       0.90      1.00      0.95         9
           3       0.88      0.78      0.82         9
           4       1.00      0.50      0.67         4

    accuracy                           0.83        66
   macro avg       0.87      0.79      0.81        66
weighted avg       0.85      0.83      0.83        66

{
    "training": {
        "device": "cuda"
    },
    "loss": {
        "type": "MultiOmicsLoss",
        "beta": 1,
        "use_focal": false,
        "focal_gamma": 2.0,
        "label_smoothing": 0.0,
        "kl_epsilon": 1e-08
    },
    "optimizer": {
        "type": "Adam",
        "lr": 2e-05,
        "betas": [
            0.9,
            0.999
        ],
        "eps": 1e-08,
        "weight_decay": 0
    }
}
